Large simulation to compare results in water between Rayleigh+FDTD and Rayleigh-alone
===
This simulation runs 308 cases with different transducers (single element, annular and phased arrays)).

### Context
BabelBrain uses the Rayleigh integral to model the acoustic sources. The acoustic field is calculated in the whole domain volume in water conditions and the layer at the boundary of the domain close to the source is used to forward propagate in the FDTD solution. We refer this as an Rayleigh+FDTD approach.

Prior to r0.3.4 of BabelBrain, BabelBrain ran twice the cycle of simulations: once with the prescence of head tissues and once in water conditions. The water conditions are very important as they are used to estimate the intensity in water conditions that is required to be applied to achieve a desired target intensity in the brain. This two-cycle approach was to ensure that the same Rayleigh+FDTD modelling conditions were applied in these calculations. 

The purpose of this study is to measure similitude metrics of the predicted acoustic fields in water conditions using Rayleigh+FDTD and Rayleigh alone. If the results converge, then that facilitates running the complete simulation in one single cycle, allowing reducing computational cost by half. The results of the Rayleigh integral in the whole domain in water conditions can then be used for the calculations of required intensity.


In [2]:
import numpy as np
import sys
import os
import time
from multiprocessing import Process, Queue
import multiprocessing
import nibabel
import time
import shutil
from glob import glob
from pprint import pprint

## Additions to path
We add the paths to BabelBrain source code

In [3]:
sys.path.append('/Users/spichardo/Documents/GitHub/BabelBrain')
sys.path.append('/Users/spichardo/Documents/GitHub/BabelBrain/BabelBrain')

In [4]:
from BabelBrain.CalculateMaskProcess import CalculateMaskProcess
from TranscranialModeling.BabelIntegrationBASE import GetSmallestSOS
#we use the Single Element sim setting
from BabelBrain.CalculateFieldProcess import CalculateFieldProcess 


StaggeredFDTD_3D_CPU loaded
StaggeredFDTD_3D_CUDA NOT loaded
StaggeredFDTD_3D_OPENCL loaded
StaggeredFDTD_3D_METAL loaded
loading /opt/homebrew/Caskroom/miniforge/base/envs/BabelJup/lib/python3.9/site-packages/BabelViscoFDTD/tools/libBabelMetal.dylib
loaded Metal <CDLL '/opt/homebrew/Caskroom/miniforge/base/envs/BabelJup/lib/python3.9/site-packages/BabelViscoFDTD/tools/libBabelMetal.dylib', handle 8be80730 at 0x13f12c790>
mkl_fft not available


## Function to generate input files from trajectories
This call the same code as in the BabelBrain's GUI step 1

In [6]:
def RunMaskGeneration(T1W,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice='6800',
                      ComputingBackend=3, 
                      Frequency=500e3,
                      BasePPW=6,
                      TxSystem='Single',
                      bUseCT=True,
                      CT_or_ZTE_input='',
                      ZTERange=[0.1,0.65],
                      HUThreshold=300.0,
                      bReuseFiles=True,
                      CTType=1):
    

    print("*"*40)
    print("*"*5+" Calculating mask.. BE PATIENT... it can take a couple of minutes...")
    print("*"*40)

    deviceName=ComputingDevice
    COMPUTING_BACKEND=ComputingBackend

    T1WIso= T1W
   
    SmallestSoS= GetSmallestSOS(Frequency,bShear=True)

    prefix = ID + '_' + TxSystem +'_%ikHz_%iPPW_' %(int(Frequency/1e3),BasePPW)

    SpatialStep=np.round(SmallestSoS/Frequency/BasePPW*1e3,3) #step of mask to reconstruct , mm
    print("Frequency, SmallestSoS, BasePPW, SpatialStep",Frequency, SmallestSoS, BasePPW,SpatialStep)

    print("Mat4Trajectory",Mat4Trajectory)

    kargs={}
    kargs['SimbNIBSDir']=simbnibs_path
    kargs['SimbNIBSType']='charm'
    kargs['CoregCT_MRI']=True
    kargs['TrajectoryType']='brainsight'
    kargs['Mat4Trajectory']=Mat4Trajectory
    kargs['T1Source_nii']=T1W
    kargs['T1Conformal_nii']=T1WIso
    kargs['SpatialStep']=SpatialStep
    kargs['Location']=[0,0,0] #This coordinate will be ignored
    kargs['prefix']=prefix
    kargs['bPlot']=False
    
    if bUseCT:
        kargs['CT_or_ZTE_input']=CT_or_ZTE_input
        kargs['CTType']=CTType
        if kargs['CTType'] in [2,3]:
            kargs['ZTERange']=ZTERange
        kargs['HUThreshold']=HUThreshold

    # Start mask generation as a separate process.
    queue=Queue()
    print(COMPUTING_BACKEND,deviceName,kargs)
    maskWorkerProcess = Process(target=CalculateMaskProcess, 
                                args=(queue,
                                     COMPUTING_BACKEND,
                                     deviceName),
                                kwargs=kargs)
    maskWorkerProcess.start()      
    # progress.
    T0=time.time()
    bNoError=True
    while maskWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False

    maskWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE calculating mask.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when generating mask!!")

## Functions to run transcranial ultrasound modelling
It calls the same code as in the BabelBrain's GUI step 2. Each transducer has slightly different set of parameters passed to the simulation functions.
 

In [7]:
def DistanceOutPlaneToFocus(FocalLength,Diameter):
    return np.sqrt(FocalLength**2-(Diameter/2)**2)
    
def RunAcousticSimSingle(T1W,
                  ID,
                  TxSystem='Single',
                  deviceName='M1',
                  Aperture = 50e-3, #in m
                  FocalLength = 50e-3, #in m
                  Frequency=500e3, #in Hz
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  TxAdjustmentZ = 0.0, 
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_Single_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    MaskData = nibabel.load(InputSim)
    FinalMask=np.flip(MaskData.get_fdata(),axis=2)

    VoxelSize=MaskData.header.get_zooms()[0]/1e3
    TargetLocation =np.array(np.where(FinalMask==5.0)).flatten()
    LineOfSight=FinalMask[TargetLocation[0],TargetLocation[1],:]
    StartSkin=np.where(LineOfSight>0)[0].min()
    DistanceFromSkin = (TargetLocation[2]-StartSkin)*VoxelSize
    

    Diameter = Aperture
    DOut=DistanceOutPlaneToFocus(FocalLength,Diameter)
    ZMax=DOut-DistanceFromSkin
    ZMaxSkin = np.round(ZMax*1e3,1)/1e3
    print('ZMaxSkin',ZMaxSkin,'DistanceFromSkin',DistanceFromSkin,'DOut',DOut)
    

    
    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= ZMaxSkin+TxAdjustmentZ  #in m
    print('DistanceFromSkin',DistanceFromSkin,'ZMaxSkin',ZMaxSkin,'TxMechanicalAdjustmentZ',TxMechanicalAdjustmentZ)
    ZIntoSkin =0.0
    CurDistance=ZMaxSkin-TxMechanicalAdjustmentZ
    if CurDistance < 0:
        ZIntoSkin = np.abs(CurDistance)

    extrasuffix='Foc%03.1f_Diam%03.1f_' %(FocalLength*1e3,Aperture*1e3)
   
    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['extrasuffix']=extrasuffix
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['Aperture']=Aperture
    kargs['FocalLength']=FocalLength
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['ZIntoSkin']=ZIntoSkin
    kargs['Frequencies']=Frequencies
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=True
    kargs['bUseRayleighForWater']= False #This will disable using Rayleigh for water calculations


    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")

def RunAcousticSimCTX500(T1W,
                  ID,
                  TxSystem='CTX_500',
                  deviceName='M1',
                  ZSteering=50e-3,
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  TxAdjustmentZ = 0.0, 
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_Single_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    MaskData = nibabel.load(InputSim)
    FinalMask=np.flip(MaskData.get_fdata(),axis=2)

    VoxelSize=MaskData.header.get_zooms()[0]/1e3
    TargetLocation =np.array(np.where(FinalMask==5.0)).flatten()
    LineOfSight=FinalMask[TargetLocation[0],TargetLocation[1],:]
    StartSkin=np.where(LineOfSight>0)[0].min()
    DistanceFromSkin = (TargetLocation[2]-StartSkin)*VoxelSize
    

    Diameter = Aperture
    DOut=52.4e-3#DistanceOutPlaneToFocus(FocalLength,Diameter)
    ZMax=DOut-DistanceFromSkin
    ZMaxSkin = np.round(ZMax*1e3,1)/1e3
    print('ZMaxSkin',ZMaxSkin,'DistanceFromSkin',DistanceFromSkin,'DOut',DOut)
    
    
    
    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= ZMaxSkin+TxAdjustmentZ  #in m
    print('DistanceFromSkin',DistanceFromSkin,'ZMaxSkin',ZMaxSkin,'TxMechanicalAdjustmentZ',TxMechanicalAdjustmentZ)
    ZIntoSkin =0.0
    CurDistance=ZMaxSkin-TxMechanicalAdjustmentZ
    if CurDistance < 0:
        ZIntoSkin = np.abs(CurDistance)

    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['ZIntoSkin']=ZIntoSkin
    kargs['ZSteering']=ZSteering
    kargs['Frequencies']=[500e3]
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=True
    kargs['bUseRayleighForWater']= False #This will disable using Rayleigh for water calculations

    print('kargs')
    pprint(kargs)

    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")


def RunAcousticSimH317(T1W,
                  ID,
                  Frequency=250e3,
                  TxSystem='H317',
                  deviceName='M1',
                  XSteering=0.0,
                  YSteering=0.0,
                  ZSteering=0.0,
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  DistanceConeToFocus = 25e-3, 
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_H317_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= 0.0 # this is overwritten later in the code
    RotationZ=0.0 
    
    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['XSteering']=XSteering
    kargs['YSteering']=YSteering
    kargs['ZSteering']=ZSteering
    kargs['RotationZ']=RotationZ
    kargs['Frequencies']=Frequencies
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bDoRefocusing']=False
    kargs['DistanceConeToFocus']=DistanceConeToFocus
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=True
    kargs['bUseRayleighForWater']= False #This will disable using Rayleigh for water calculations
    kargs['bDryRun']=False
    kargs['MultiPoint']=None

    print('kargs')
    pprint(kargs)

    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")

def RunAcousticSimREMOPD(T1W,
                  ID,
                  Frequency=300e3,
                  TxSystem='REMOPD',
                  deviceName='M1',
                  XSteering=0.0,
                  YSteering=0.0,
                  ZSteering=0.0,
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_H317_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= 0.0 # this is overwritten later in the code
    
    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['XSteering']=XSteering
    kargs['YSteering']=YSteering
    kargs['ZSteering']=ZSteering
    kargs['Frequencies']=Frequencies
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bDoRefocusing']=False
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=True
    kargs['bUseRayleighForWater']= False #This will disable using Rayleigh for water calculations
    kargs['bDryRun']=False
    

    print('kargs')
    pprint(kargs)

    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")

#  Dataset
We use the dataset corresponding to SDR=0.55 available in the BabelBrain's Zenodo repository. Download it and keep note of the file path to it.

https://doi.org/10.5281/zenodo.7894431

# Simulation settings
Simulations are run for four transducers available in BabelBrain: single element concave (with differnt focal lengths, apertures, and frequencies), CTX_500 (annular array), H317 (large 128-element concave phased array) and REMOPD (256-element flat array).

We keep only the Nifti files for the results of Rayleigh alone and FDTD.

# VERY IMPORTANT
Run always **all the cells** as there is a counter of the number of case. Simulations will skip cases that have been completed.

In [11]:
#specify the path to location where 
BasePath='/Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/'

#Specify a path where the simulation cases will be saved
Cases_dir= BasePath+'Cases'
if not os.path.isdir(Cases_dir):
    os.mkdir(Cases_dir)

T1W = 'T1W.nii.gz'
CT='CT.nii.gz'
simbnibs_path=BasePath+'m2m_SDR_0p55'
Mat4Trajectory =  './DEEP.txt'
ID = 'DEEP'

#ID of GPU device to run the simulations
ComputingDevice='M1'
ComputingBackend=3 # 1 for CUDA, 2 for OpenCL, 3 for Metal

## Simulations for Single Tx
We test different frequencies, PPWs, apertures, focal lenths and repositions of the transducer.

In [9]:
TxSystem='Single'
FrequenciesPPWs=[[250e3,[6,9]],[500e3,[6,9]],[750e3,[6]]]
Apertures = [50e-3,60e-3,70e-3]
FocalLengths = [50e-3,60e-3,70e-3]
zLengthBeyonFocalPointWhenNarrow = 80e-3
zAdjustments=[-10e-3,0.0,10e-3]

In [13]:
#we first create the simulation domains (Step 1 in BabelBrain)
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        prevfile=os.path.join(BasePath,ID+'_'+TxSystem+'_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
        if os.path.isfile(prevfile):
            print('skipping ', prevfile)
            continue
        RunMaskGeneration(BasePath+T1W,
                          ID,
                          simbnibs_path,
                          Mat4Trajectory,
                          ComputingDevice=ComputingDevice,
                          ComputingBackend=ComputingBackend, #Metal
                          Frequency=Frequency,
                          BasePPW=PPW,
                          TxSystem=TxSystem,
                          bUseCT=True,
                          CT_or_ZTE_input=BasePath+CT,
                          ZTERange=[0.1,0.65],
                          HUThreshold=300.0,
                          CTType=1)
    

skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_Single_250kHz_6PPW_BabelViscoInput.nii.gz
skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_Single_250kHz_9PPW_BabelViscoInput.nii.gz
skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_Single_500kHz_6PPW_BabelViscoInput.nii.gz
skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_Single_500kHz_9PPW_BabelViscoInput.nii.gz
skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_Single_750kHz_6PPW_BabelViscoInput.nii.gz


In [ ]:
# We initialize the global counter of cases
GLOBAL_CASE_COUNTER =-1
#we run the simulations corresponding step 2 in BabelBrain
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        for zAdjust in zAdjustments:
            for Aperture in Apertures:
                for FocalLength in FocalLengths:
                    GLOBAL_CASE_COUNTER+=1
                   
                    infix = ID+'_Single_%ikHz_%iPPW_Foc%03.1f_Diam%03.1f' \
                                       %(int(Frequency/1e3),PPW,FocalLength*1e3,Aperture*1e3)
                    prefix=os.path.join(BasePath,infix)
                    PrevDate=None
                    PrevDateRayleigh=None

                    FDTDName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'FDTD_ZAdj_%03.1f_' %(zAdjust*1e3) + infix +'.nii.gz'
                    RayleighName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'Rayleigh_ZAdj_%03.1f_' %(zAdjust*1e3)  + infix +'.nii.gz'
                    if os.path.isfile(FDTDName) and  os.path.isfile(RayleighName):
                        print('skipping case',GLOBAL_CASE_COUNTER)
                        continue          
                    
                    FDTDResults =prefix+'_Water_FullElasticSolution_Sub.nii.gz'
                    RayleighResults =prefix+'_RayleighFreeWater_Sub.nii.gz'

                    if os.path.isfile(FDTDResults):
                        PrevDate=time.ctime(os.path.getmtime(FDTDResults))
                    if os.path.isfile(RayleighResults):
                        PrevDateRayleigh=time.ctime(os.path.getmtime(RayleighResults))
                    RunAcousticSimSingle(BasePath+T1W,
                                   ID,
                                   TxSystem=TxSystem,
                                  deviceName=ComputingDevice,
                                  Aperture = Aperture, #in m
                                  FocalLength = FocalLength, #in m
                                  Frequency=Frequency, #in Hz
                                  PPW=PPW,
                                  TxAdjustmentZ=zAdjust,
                                  COMPUTING_BACKEND=3,
                                  zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow, #distance to run simulation beyond target
                                  bUseCT = False)
                    if PrevDate is not None:
                        assert(time.ctime(os.path.getmtime(FDTDResults))>PrevDate)
                    if PrevDateRayleigh is not None:
                        assert(time.ctime(os.path.getmtime(RayleighResults))>PrevDateRayleigh)
                    
                    
                    shutil.move(FDTDResults,FDTDName)
                    shutil.move(RayleighResults,RayleighName)
                    #we remove all the extra files we do not need anymore
                    for f in glob(prefix+"*"):
                        os.remove(f)
                    


## Simulations for CTX_500
This device operates at 500 kHz. We test different steering along acoustic axis, and repositions of the transducer.

In [16]:
PPWs=[6,9]
TxSystem='CTX_500'
ZSteerings = np.arange(-20,30,12.5)*1e-3
zLengthBeyonFocalPointWhenNarrow = 40e-3
zAdjustments=[-10e-3,0.0,10e-3]

In [17]:
Frequency = 500e3
for PPW in PPWs:
    prevfile=os.path.join(BasePath,ID+'_'+TxSystem+'_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    if os.path.isfile(prevfile):
        print('skipping ', prevfile)
        continue
    RunMaskGeneration(BasePath+T1W,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice=ComputingDevice,
                      ComputingBackend=ComputingBackend, #Metal
                      Frequency=Frequency,
                      BasePPW=PPW,
                      TxSystem=TxSystem,
                      bUseCT=True,
                      CT_or_ZTE_input=BasePath+CT,
                      ZTERange=[0.1,0.65],
                      HUThreshold=300.0,
                      CTType=1)

skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_CTX_500_500kHz_6PPW_BabelViscoInput.nii.gz
skipping  /Users/spichardo/Library/CloudStorage/OneDrive-UniversityofCalgary/GDrive/ReplaceWaterBabel/SDR0p55/DEEP_CTX_500_500kHz_9PPW_BabelViscoInput.nii.gz


In [18]:
for PPW in PPWs:
    for zAdjust in zAdjustments:
        for ZSteering in ZSteerings:
            GLOBAL_CASE_COUNTER+=1
           
            infix = ID+'_CTX_500_500kHz_%iPPW' %(PPW) 
            prefix=os.path.join(BasePath,infix)
            PrevDate=None
            PrevDateRayleigh=None

            FDTDName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'FDTD_ZAdj_%03.1f_' %(zAdjust*1e3) + infix +'_ZSteering_%03.1f' % (ZSteering*1e3)+'.nii.gz'
            RayleighName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'Rayleigh_ZAdj_%03.1f_' %(zAdjust*1e3) + infix +'_ZSteering_%03.1f' % (ZSteering*1e3) +'.nii.gz'

            if os.path.isfile(FDTDName) and  os.path.isfile(RayleighName):
                print('skipping case',GLOBAL_CASE_COUNTER)
                continue  
            
           
            FDTDResults =prefix+'_Water_FullElasticSolution_Sub.nii.gz'
            RayleighResults =prefix+'_RayleighFreeWater_Sub.nii.gz'

            if os.path.isfile(FDTDResults):
                PrevDate=time.ctime(os.path.getmtime(FDTDResults))
            if os.path.isfile(RayleighResults):
                PrevDateRayleigh=time.ctime(os.path.getmtime(RayleighResults))
            RunAcousticSimCTX500(BasePath+T1W,
                           ID,
                           TxSystem=TxSystem,
                          deviceName=ComputingDevice,
                          ZSteering=ZSteering,
                          PPW=PPW,
                          TxAdjustmentZ=zAdjust,
                          COMPUTING_BACKEND=3,
                          zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow, #distance to run simulation beyond target
                          bUseCT = False)
            if PrevDate is not None:
                assert(time.ctime(os.path.getmtime(FDTDResults))>PrevDate)
            if PrevDateRayleigh is not None:
                assert(time.ctime(os.path.getmtime(RayleighResults))>PrevDateRayleigh)
            
            
            shutil.move(FDTDResults,FDTDName)
            shutil.move(RayleighResults,RayleighName)
            ###we remove all the extra files we do not need anymore
                    


skipping case 135
skipping case 136
skipping case 137
skipping case 138
skipping case 139
skipping case 140
skipping case 141
skipping case 142
skipping case 143
skipping case 144
skipping case 145
skipping case 146
skipping case 147
skipping case 148
skipping case 149
skipping case 150
skipping case 151
skipping case 152
skipping case 153
skipping case 154
skipping case 155
skipping case 156
skipping case 157
skipping case 158


## Run Simulations for H317

In [ ]:
TxSystem='H317'
FrequenciesPPWs=[[250e3,[6,9]],[750e3,[6]]]
XSteerings = [0.0, 10e-3]
YSteerings = [0.0, 10e-3]
ZSteerings = [-10e-3,0.0,10e-3,20e-3]
zLengthBeyonFocalPointWhenNarrow = 40e-3
DistancesConeToFocus = [30e-3,65e-3]

In [ ]:
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        prevfile=os.path.join(BasePath,ID+'_'+TxSystem+'_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
        if os.path.isfile(prevfile):
            print('skipping ', prevfile)
            continue
        RunMaskGeneration(BasePath+T1W,
                              ID,
                              simbnibs_path,
                              Mat4Trajectory,
                              ComputingDevice=ComputingDevice,
                              ComputingBackend=ComputingBackend, #Metal
                              Frequency=Frequency,
                              BasePPW=PPW,
                              TxSystem=TxSystem,
                              bUseCT=True,
                              CT_or_ZTE_input=BasePath+CT,
                              ZTERange=[0.1,0.65],
                              HUThreshold=300.0,
                              CTType=1)

In [ ]:
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        for XSteering in XSteerings:
            for YSteering in YSteerings:
                for ZSteering in ZSteerings:
                    for DistanceConeToFocus in DistancesConeToFocus:
                        GLOBAL_CASE_COUNTER+=1
                       
                        infix = ID+'_H317_%ikHz_%iPPW' %(int(Frequency/1e3),PPW)
                        prefix=os.path.join(BasePath,infix)
                        
                        PrevDate=None
                        PrevDateRayleigh=None
    
                        postfix = 'ConeDistance_%03.1f_' %(DistanceConeToFocus*1e3) + infix +'_XSteer_%03.1f' % (XSteering*1e3) +'_YSteer_%03.1f' % (YSteering*1e3) +'_ZSteer_%03.1f' % (ZSteering*1e3)+'.nii.gz'
    
                        FDTDName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'FDTD_' +postfix
                        RayleighName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'Rayleigh_'+postfix
                        if os.path.isfile(FDTDName) and  os.path.isfile(RayleighName):
                            print('skipping case',GLOBAL_CASE_COUNTER)
                            continue          
                        
                        FDTDResults =prefix+'_Water_FullElasticSolution_Sub.nii.gz'
                        RayleighResults =prefix+'_RayleighFreeWater_Sub.nii.gz'
    
                        if os.path.isfile(FDTDResults):
                            PrevDate=time.ctime(os.path.getmtime(FDTDResults))
                        if os.path.isfile(RayleighResults):
                            PrevDateRayleigh=time.ctime(os.path.getmtime(RayleighResults))
                        RunAcousticSimH317(BasePath+T1W,
                                       ID,
                                       TxSystem=TxSystem,
                                       deviceName=ComputingDevice,
                                       Frequency=Frequency, #in Hz
                                       PPW=PPW,
                                       XSteering=XSteering,
                                       YSteering=YSteering,
                                       ZSteering=ZSteering,
                                       DistanceConeToFocus=DistanceConeToFocus,
                                       COMPUTING_BACKEND=3,
                                       zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow, #distance to run simulation beyond target
                                       bUseCT = True)
                        if PrevDate is not None:
                            assert(time.ctime(os.path.getmtime(FDTDResults))>PrevDate)
                        if PrevDateRayleigh is not None:
                            assert(time.ctime(os.path.getmtime(RayleighResults))>PrevDateRayleigh)
                        
                        
                        shutil.move(FDTDResults,FDTDName)
                        shutil.move(RayleighResults,RayleighName)

## Run Simulations for REMOPD



In [ ]:
TxSystem='REMOPD'
FrequenciesPPWs=[[250e3,[6,9]]]
XSteerings = [0.0, 10e-3,20e-3]
YSteerings = [0.0, 10e-3,20e-3]
ZSteerings = [40e-3,60e-3,80e-3]
zLengthBeyonFocalPointWhenNarrow = 60e-3

In [ ]:
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        prevfile=os.path.join(BasePath,ID+'_'+TxSystem+'_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
        if os.path.isfile(prevfile):
            print('skipping ', prevfile)
            continue
        RunMaskGeneration(BasePath+T1W,
                              ID,
                              simbnibs_path,
                              Mat4Trajectory,
                              ComputingDevice=ComputingDevice,
                              ComputingBackend=ComputingBackend, #Metal
                              Frequency=Frequency,
                              BasePPW=PPW,
                              TxSystem=TxSystem,
                              bUseCT=True,
                              CT_or_ZTE_input=BasePath+CT,
                              ZTERange=[0.1,0.65],
                              HUThreshold=300.0,
                              CTType=1)

In [ ]:
for FrequencyPPW in FrequenciesPPWs:
    Frequency = FrequencyPPW[0]
    for PPW in FrequencyPPW[1]:
        for XSteering in XSteerings:
            for YSteering in YSteerings:
                for ZSteering in ZSteerings:
                
                    GLOBAL_CASE_COUNTER+=1
                   
                    infix = ID+'_'+TxSystem+'_%ikHz_%iPPW' %(int(Frequency/1e3),PPW)
                    prefix=os.path.join(BasePath,infix)
                    
                    PrevDate=None
                    PrevDateRayleigh=None

                    postfix = infix +'_XSteer_%03.1f' % (XSteering*1e3) +'_YSteer_%03.1f' % (YSteering*1e3) +'_ZSteer_%03.1f' % (ZSteering*1e3)+'.nii.gz'

                    FDTDName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'FDTD_' +postfix
                    RayleighName = Cases_dir + os.sep + 'CASE_%04i_' %(GLOBAL_CASE_COUNTER) + 'Rayleigh_'+postfix
                    if os.path.isfile(FDTDName) and  os.path.isfile(RayleighName):
                        print('skipping case',GLOBAL_CASE_COUNTER)
                        continue          
                    
                    FDTDResults =prefix+'_Water_FullElasticSolution_Sub.nii.gz'
                    RayleighResults =prefix+'_RayleighFreeWater_Sub.nii.gz'

                    if os.path.isfile(FDTDResults):
                        PrevDate=time.ctime(os.path.getmtime(FDTDResults))
                    if os.path.isfile(RayleighResults):
                        PrevDateRayleigh=time.ctime(os.path.getmtime(RayleighResults))
                    RunAcousticSimREMOPD(BasePath+T1W,
                                   ID,
                                   TxSystem=TxSystem,
                                   deviceName=ComputingDevice,
                                   Frequency=Frequency, #in Hz
                                   PPW=PPW,
                                   XSteering=XSteering,
                                   YSteering=YSteering,
                                   ZSteering=ZSteering,
                                   COMPUTING_BACKEND=3,
                                   zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow, #distance to run simulation beyond target
                                   bUseCT = True)
                    if PrevDate is not None:
                        assert(time.ctime(os.path.getmtime(FDTDResults))>PrevDate)
                    if PrevDateRayleigh is not None:
                        assert(time.ctime(os.path.getmtime(RayleighResults))>PrevDateRayleigh)
                    
                    
                    shutil.move(FDTDResults,FDTDName)
                    shutil.move(RayleighResults,RayleighName)